# Multi-Agent Researcher with MCP Support

This notebook demonstrates how to use the multi-agent research system with Model Context Protocol (MCP) support. It follows the same patterns as the original multi_agent.ipynb but adds MCP integration.

In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/Users/bartolli/Projects/playgound/open_deep_research_05/open_deep_research/src


In [2]:
import uuid 
import os
import open_deep_research   
print(open_deep_research.__version__) 
from IPython.display import Image, display, Markdown
from langgraph.checkpoint.memory import MemorySaver
import logging

# Set up logging for better visibility
logging.basicConfig(level=logging.INFO)

0.0.15


## Configure MCP Server

We'll use the example MCP server that provides patient data.

In [3]:
# Get the path to the example_mcp_server.py 
import pathlib
repo_root = pathlib.Path(".").resolve()
example_server_path = str(repo_root / "open_deep_research" / "example_mcp_server.py")

# Check if file exists
print(f"Example MCP server path: {example_server_path}")
print(f"File exists: {pathlib.Path(example_server_path).exists()}")

# Try alternate locations if not found
if not pathlib.Path(example_server_path).exists():
    alternate_path = str(repo_root / "open_deep_research" / "example_mcp_server.py")
    print(f"Trying alternate path: {alternate_path}")
    print(f"Alternate path exists: {pathlib.Path(alternate_path).exists()}")
    if pathlib.Path(alternate_path).exists():
        example_server_path = alternate_path

# Define MCP server configuration
mcp_servers_config = {
    "patient_data": {
        "transport": "stdio",
        "command": "python",
        "args": [example_server_path],
    }
}

Example MCP server path: /Users/bartolli/Projects/playgound/open_deep_research_05/open_deep_research/src/open_deep_research/example_mcp_server.py
File exists: True


## Build the Graph with MCP Support

Instead of directly using supervisor_g.compile(), we'll use the build_graph_with_mcp function.

In [ ]:
# Import the build_graph_with_mcp function
from open_deep_research.multi_agent_mcp import supervisor_builder

# Create a MemorySaver for checkpointing
checkpointer = MemorySaver()

# Create base configuration with proper model specifications
base_config = {
    "thread_id": str(uuid.uuid4()),
    "search_api": "tavily",
    # "supervisor_model": "anthropic:claude-3-5-sonnet-latest",  # Use appropriate model
    # "researcher_model": "anthropic:claude-3-5-sonnet-latest",  # Use appropriate model
    "mcp_servers": mcp_servers_config,
}

# This is a notebook-specific adaptation that combines build_graph_with_mcp with checkpointer
async def build_graph_with_checkpointer(cfg):
    # First get the MCP configuration but don't create the manager yet (like in build_graph_with_mcp)
    from open_deep_research.configuration import Configuration
    
    cfg = cfg or {}
    configurable = Configuration.from_runnable_config(cfg)
    mcp_servers = configurable.mcp_servers
    
    # Store MCP servers config in a special key (same as build_graph_with_mcp)
    if mcp_servers:
        print(f"MCP servers configured: {list(mcp_servers.keys())}")
        if "configurable" not in cfg:
            cfg["configurable"] = {}
        cfg["configurable"]["_mcp_servers_config"] = mcp_servers
    
    # Now compile with checkpointer (different from build_graph_with_mcp)
    g = supervisor_builder.compile(name="research_team", checkpointer=checkpointer)
    
    # Apply config after compilation
    g = g.with_config(cfg)
    
    # Return in the same format as build_graph_with_mcp
    return g, None, cfg

# Build the graph using our adapted function
agent, _, thread_config = await build_graph_with_checkpointer(base_config)

MCP servers configured: ['patient_data']


## Visualize the Graph Structure

This shows how supervisor and research agents are connected in the workflow.

In [ ]:
display(Image(agent.get_graph(xray=1).draw_mermaid_png(max_retries=5)))

## Run the Multi-Agent System

We'll use a focused query that leverages the patient data from the MCP server.

In [5]:
# Define the research topic as a user message
msg = [{"role": "user", "content": "Check what we know about patient with id P001 and conduct research on the latest treatment approaches and management strategies for these conditions."}]

# Create a properly structured config dictionary for invoking the agent
access_config = {
    "configurable": {
        "thread_id": base_config["thread_id"],
        "mcp_servers": base_config["mcp_servers"],
    }
}

# Run the multi-agent workflow with the properly structured configuration
response = await agent.ainvoke({"messages": msg}, config=access_config)

INFO:open_deep_research.multi_agent_mcp:Initializing MCP manager with 1 servers
INFO:open_deep_research.mcp_integration:Creating MCP manager with servers: ['patient_data']
INFO:open_deep_research.mcp_integration:Initializing MCP client with servers: ['patient_data']
INFO:open_deep_research.mcp_integration:Server patient_data config: transport=stdio
INFO:open_deep_research.mcp_integration:  Command: python /Users/bartolli/Projects/playgound/open_deep_research_05/open_deep_research/src/open_deep_research/example_mcp_server.py
INFO:open_deep_research.mcp_integration:Successfully loaded 3 tools from MCP servers
INFO:open_deep_research.mcp_integration:Tool 1: get_patient_vitals - 
Use this tool to retrieve current vital signs for a patient for your research.

Args:
    patient_id: Unique identifier for the patient
    
Returns:
    String containing vital measurements information

INFO:open_deep_research.mcp_integration:  Arguments: {'patient_id': {'title': 'Patient Id', 'type': 'string'}}


## View the Supervisor's Response

In [6]:
# Create a properly structured config dictionary for state access
access_config = {
    "configurable": {
        "thread_id": base_config["thread_id"],
        "mcp_servers": base_config["mcp_servers"],
    }
}

# Then use this properly structured config to access the state
messages = agent.get_state(access_config).values['messages']
messages[-1].pretty_print()

================================== Ai Message ==================================

Here is what I have learned from your patient data and my research:

**Patient P001** has:
- Type 2 Diabetes Mellitus (moderate, managed)
- Essential Hypertension (mild, active)

The latest evidence-based treatment approaches and management strategies for both conditions for adults emphasize a multifactorial, patient-centered management plan, including the following major points:

**For Type 2 Diabetes Mellitus:**
- Current guidelines (ADA 2023, NICE 2022, and others) recommend starting with comprehensive lifestyle interventions (diet, physical activity, weight loss) as the foundation.
- Pharmacological therapies are tailored based on the patient's comorbidities and risk profile. Metformin remains first-line, but newer agents (SGLT-2 inhibitors, GLP-1 receptor agonists) are favored—especially if cardiovascular or renal risk is present.
- Shared, person-centered decision-making is emphasized in choosing th

## Add Clarification if Needed

If the supervisor asks for clarification, we can add more details.

In [7]:
# Create a follow-up message with your clarification
follow_up_msg = [
    {"role": "user", "content": "Focus on the latest integrated management approaches and patient self-management. The report is for patient self-education. Adjust narrative accordingly. Target audience: average."},
]

# Create a properly structured config dictionary for state access
access_config = {
    "configurable": {
        "thread_id": base_config["thread_id"],
        "mcp_servers": base_config["mcp_servers"],
    }
}

# Run the multi-agent workflow again with your clarification
response = await agent.ainvoke({"messages": follow_up_msg}, config=access_config)

# Check the response
messages = agent.get_state(access_config).values['messages']
messages[-1].pretty_print()

INFO:open_deep_research.multi_agent_mcp:Initializing MCP manager with 1 servers
INFO:open_deep_research.mcp_integration:Creating MCP manager with servers: ['patient_data']
INFO:open_deep_research.mcp_integration:Initializing MCP client with servers: ['patient_data']
INFO:open_deep_research.mcp_integration:Server patient_data config: transport=stdio
INFO:open_deep_research.mcp_integration:  Command: python /Users/bartolli/Projects/playgound/open_deep_research_05/open_deep_research/src/open_deep_research/example_mcp_server.py
INFO:open_deep_research.mcp_integration:Successfully loaded 3 tools from MCP servers
INFO:open_deep_research.mcp_integration:Tool 1: get_patient_vitals - 
Use this tool to retrieve current vital signs for a patient for your research.

Args:
    patient_id: Unique identifier for the patient
    
Returns:
    String containing vital measurements information

INFO:open_deep_research.mcp_integration:  Arguments: {'patient_id': {'title': 'Patient Id', 'type': 'string'}}


================================== Ai Message ==================================

Thank you for clarifying! Here’s what I understand based on our exchanges and the research:

- The report should focus on **integrated approaches** to managing both type 2 diabetes and hypertension together, reflecting the latest guidelines.
- It should emphasize **patient self-management strategies**, empowering the individual to play an active role in their own care.
- The narrative should be **clear and accessible** for someone without a medical background.
- The goal is **self-education** for patient P001 (or someone with the same conditions).
- Priorities: practical day-to-day management, understanding treatment options, and effective collaboration with healthcare providers.

**Before moving forward:**  
- Should I include information or tips about digital health tools (apps, telehealth, glucose monitors, etc.) for self-management and monitoring?
- Are there language or cultural needs I should be awa

In [ ]:
# Create a follow-up message with your clarification
confirmation_msg = [
    {"role": "user", "content": "I confirm the scope. Adjust narrative for average audience. Focus on personal lifestyle/self-management day in the life."}
]

# Create a properly structured config dictionary for state access
access_config = {
    "configurable": {
        "thread_id": base_config["thread_id"],
        "mcp_servers": base_config["mcp_servers"],
    }
}

# Run the multi-agent workflow again with your clarification
response = await agent.ainvoke({"messages": confirmation_msg}, config=access_config)

# Check the response
messages = agent.get_state(access_config).values['messages']
messages[-1].pretty_print()

INFO:open_deep_research.multi_agent_mcp:Initializing MCP manager with 1 servers
INFO:open_deep_research.mcp_integration:Creating MCP manager with servers: ['patient_data']
INFO:open_deep_research.mcp_integration:Initializing MCP client with servers: ['patient_data']
INFO:open_deep_research.mcp_integration:Server patient_data config: transport=stdio
INFO:open_deep_research.mcp_integration:  Command: python /Users/bartolli/Projects/playgound/open_deep_research_05/open_deep_research/src/open_deep_research/example_mcp_server.py
INFO:open_deep_research.mcp_integration:Successfully loaded 3 tools from MCP servers
INFO:open_deep_research.mcp_integration:Tool 1: get_patient_vitals - 
Use this tool to retrieve current vital signs for a patient for your research.

Args:
    patient_id: Unique identifier for the patient
    
Returns:
    String containing vital measurements information

INFO:open_deep_research.mcp_integration:  Arguments: {'patient_id': {'title': 'Patient Id', 'type': 'string'}}


================================== Ai Message ==================================

Thank you for guiding the report to completion! 

You now have a well-structured, self-education guide for patients living with both type 2 diabetes and hypertension, featuring:

---

# Integrated Self-Management for Diabetes and Hypertension: A Practical Guide for Everyday Life

Living with type 2 diabetes and high blood pressure can feel overwhelming, but with the right knowledge and daily habits, you can take strong, confident steps toward better health. This report is designed just for you, using clear language to explain the newest ways to manage both conditions together. Whether you’re just starting out or looking to refresh your skills, you’ll find real-world tips for putting doctors’ advice into practice, making medication routines easier, and using up-to-date tools and technology for home care. You’ll also learn simple strategies for eating, staying active, tracking your progress, and looking aft

/Users/bartolli/Projects/playgound/open_deep_research_05/open_deep_research/.venv/lib/python3.13/site-packages/langgraph/checkpoint/serde/jsonplus.py:526: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return cls(**tup[2])


## View the Final Report

Once all sections are completed, introduction and conclusion are added, the final report will be available.

In [ ]:
# Create the properly structured access config
access_config = {
    "configurable": {
        "thread_id": base_config["thread_id"],
        "mcp_servers": base_config["mcp_servers"],
    }
}

# Get the current state
state = agent.get_state(access_config).values

# Check if the final report is available
if 'final_report' in state and state['final_report']:
    print("✅ Final report is complete:")
    display(Markdown(state['final_report']))
else:
    print("⏳ Report is still being generated. Showing available content so far:")
    
    # Construct a partial report from available components
    report_parts = []
    
    # Add introduction if available
    if 'final_report' in state and state['final_report']:
        report_parts.append(state['final_report'])  # This might contain just the intro
    
    # Add completed sections if available
    if 'completed_sections' in state and state['completed_sections']:
        print(f"- {len(state['completed_sections'])} sections completed")
        for section in state['completed_sections']:
            report_parts.append(section.content)
    
    # Display what we have so far
    if report_parts:
        display(Markdown("\n\n".join(report_parts)))
    else:
        print("No content available yet.")
        
    # Show message count for debugging
    print(f"\nCurrent message count: {len(state['messages'])}")

✅ Final report is complete:


# # Integrated Self-Management for Diabetes and Hypertension: A Practical Guide for Everyday Life

Living with type 2 diabetes and high blood pressure can feel overwhelming, but with the right knowledge and daily habits, you can take strong, confident steps toward better health. This report is designed just for you, using clear language to explain the newest ways to manage both conditions together. Whether you’re just starting out or looking to refresh your skills, you’ll find real-world tips for putting doctors’ advice into practice, making medication routines easier, and using up-to-date tools and technology for home care. You’ll also learn simple strategies for eating, staying active, tracking your progress, and looking after your mental and emotional wellbeing. The goal: to empower you to live a healthier life, one day at a time, using an approach that works for your daily routine.

## Medications: Understanding Common Treatments and How They Work Together for Diabetes and Hypertension

- **Diabetes medications:**
  - *Metformin* is often the first choice; lowers blood sugar and protects the heart.
  - *GLP-1 agonists* (like semaglutide) and *SGLT-2 inhibitors* (like empagliflozin) help control blood sugar, reduce weight, and protect heart/kidney health. Many adults with type 2 diabetes may benefit from these newer medicines.
- **Hypertension (high blood pressure) medications:**
  - *ACE inhibitors* (ending in "-pril", e.g., lisinopril) and *ARBs* (ending in "-sartan", e.g., losartan) relax blood vessels—especially important for people with diabetes because they help protect the kidneys.
  - *Calcium channel blockers* (e.g., amlodipine), *diuretics* (water pills), and *beta-blockers* (ending in "-olol") are also common.
- **Using them safely together:**
  - Combining these drugs is common and can lower risk for heart attack, stroke, and kidney damage.
  - Always take medicines as prescribed, and inform your doctor of all medications to avoid harmful interactions.
  - Watch for side effects (e.g., cough from ACE inhibitors or swelling from CCBs).

### Sources
1. https://www.diabetescarecommunity.ca/living-well-with-diabetes-articles/common-blood-pressure-medications-for-people-with-diabetes/
2. https://www.cdc.gov/diabetes/data-research/research/new-diabetes-medicines.html
3. https://www.healthgrades.com/right-care/high-blood-pressure/high-blood-pressure-medication

## Day-to-Day Self-Management: At Home Strategies

Effective home self-management is vital for controlling chronic conditions and improving quality of life. Evidence-based strategies include:

- **Healthy Eating:** Emphasize fruits, vegetables, whole grains, and lean proteins; limit added sugars, salt, and saturated fat. The Dietary Guidelines for Americans and resources like MyPlate can guide balanced meal planning.
- **Physical Activity:** Adults should aim for 150 minutes/week of moderate-intensity activity and strength exercises twice weekly. Even small increases in activity support function and mental health.
- **Medication Routines:** Take medications as prescribed, at the correct times and dosages. If unsure, consult your doctor or pharmacist for clarity.
- **Monitoring Numbers:** Regularly track key health measures (e.g., blood pressure, blood sugar) as directed. Report concerning trends or symptoms promptly to your care team.
- **Skills and Support:** Develop self-management skills such as goal setting, problem-solving, decision-making, and building a partnership with health care providers. Peer support groups and structured programs can offer guidance.

These practices together can reduce complications, lower healthcare use, and improve well-being.

### Sources
1. https://www.cdc.gov/chronic-disease/living-with/index.html
2. https://pmc.ncbi.nlm.nih.gov/articles/PMC6684026/
3. https://pmc.ncbi.nlm.nih.gov/articles/PMC6675068/

## Practical Self-Monitoring: How to Use Home Blood Sugar and Blood Pressure Tools

- **Blood Sugar Monitoring:** Use a lancet to prick the fingertip, place a drop of blood on a testing strip, and insert it into the glucose monitor. Follow your device’s instructions, track readings (before/after meals, as advised), and maintain clean equipment. Results outside your personalized target range or repeated abnormal readings require consulting your healthcare provider. Warning symptoms include confusion, dizziness, or persistent high/low readings.
- **Blood Pressure Monitoring:** Sit quietly for 5 minutes, feet flat, arm at heart level. Wrap the cuff snugly and start the monitor. Record both systolic and diastolic numbers, ideally at the same time daily. Seek medical attention if readings repeatedly exceed 180/120 mmHg or fall below 90/60 mmHg, or if you experience chest pain, shortness of breath, or confusion.
- **When to Get Help:** Any persistent abnormal results, sudden changes, or symptoms (e.g., fainting, chest pain, severe headaches) warrant immediate medical evaluation.

### Sources
1. https://www.medicalnewstoday.com/articles/317224
2. https://www.healthline.com/health/type-2-diabetes/blood-glucose-monitoring
3. https://gethealthyclarkcounty.org/best-practices-for-accurate-blood-pressure-readings-at-home/

## Technology for Self-Care: Using Apps, Devices, and Digital Tools for Daily Health Management

Modern technology offers easy and effective ways for people to manage their health each day:

- **Wearable devices** (smartwatches, fitness trackers) monitor steps, heart rate, sleep, or even blood sugar in real time, providing instant feedback to help build healthy habits and spot potential problems early.
- **Health apps** let users track exercise, nutrition, medication, or symptoms, encouraging consistency, providing reminders, and delivering personalized suggestions.
- **Remote monitoring** allows information from wearables and apps to be shared with healthcare providers, supporting early intervention and tailored advice.
- **Digital platforms** support everything from virtual doctor visits to guided meditations, making expert help and wellness resources more accessible.

Facts:
- Around 20% of US adults own a smart wearable device; using wearables increases daily activity by about 1,300 steps on average.
- Research shows these tools foster user empowerment, better self-management of chronic conditions, and positive behavior changes.
- Privacy and device accuracy are important—choose reputable apps and devices and consult health professionals when needed.

### Sources
1. https://www.hcplive.com/view/how-digital-health-tools-ai-and-wearables-are-transforming-care-and-self-management
2. https://pmc.ncbi.nlm.nih.gov/articles/PMC9330198/
3. https://www.uclahealth.org/news/article/7-ways-wearable-technology-can-help-you-reach-your-health

## Motivation, Mental Health, and Overcoming Common Barriers

- **Understand Your Motivation:** Motivation can be intrinsic (personal satisfaction) or extrinsic (external rewards); identifying what drives you increases goal adherence.
- **Set Specific, Meaningful Goals:** Use strategies like S.M.A.R.T. goals and connect actions to core values for clarity and resilience.
- **Common Barriers:** Fatigue, fear of failure, low self-efficacy, fixed mindset, unclear goals, and lack of support frequently undermine progress.
- **Mental Health Matters:** Depression, anxiety, and burnout can impede motivation—address these by building routines, practicing self-compassion, and seeking help when needed.
- **Evidence-Based Strategies:**
  - Embrace a growth mindset—view setbacks as learning opportunities.
  - Visualize success and track progress to maintain momentum.
  - Create support systems and accountability networks for encouragement.
  - Practice self-compassion and celebrate small successes to sustain motivation.

Protecting your mental health and applying these proven methods can help you overcome obstacles and stay motivated toward your goals.

### Sources
1. https://psyforu.com/stay-on-track-the-psychological-foundations-of-motivation-and-success/
2. https://psychnewsdaily.com/motivation-strategies/
3. https://www.psychologytoday.com/us/blog/being-your-best-self/202411/12-common-barriers-that-get-in-the-way-of-your-motivation

## ## Conclusion

Taking charge of both type 2 diabetes and high blood pressure is possible—one step and one day at a time. By combining lifestyle changes, practical self-monitoring, and the right medications, you can reduce your risks and live well. Technology offers extra support, and working closely with your healthcare team ensures you stay on track. Don’t forget: your mental and emotional health matter just as much as your lab results. Keep asking questions, learning, and reaching for small wins each day. Every healthy choice adds up. You’ve got this!

## Optional: Continue the Research Process

If the report isn't complete, you can continue with further interactions.

In [ ]:
# Optional: Request elaboration on a specific section or topic
# Uncomment and modify when you want to explore a specific area in more detail

# Create a list of possible section names if they exist in the state
access_config = {
    "configurable": {
        "thread_id": base_config["thread_id"],
        "mcp_servers": base_config["mcp_servers"],
    }
}

# Check if there are completed sections to reference
if 'completed_sections' in agent.get_state(access_config).values:
    section_names = [section.name for section in agent.get_state(access_config).values['completed_sections']]
    print("Available sections that could be elaborated:")
    for i, name in enumerate(section_names):
        print(f"{i+1}. {name}")



Available sections that could be elaborated:
1. Medications: Understanding Common Treatments and How They Work Together for Diabetes and Hypertension
2. Day-to-Day Self-Management: At Home Strategies
3. Practical Self-Monitoring: How to Use Home Blood Sugar and Blood Pressure Tools
4. Technology for Self-Care: Using Apps, Devices, and Digital Tools for Daily Health Management
5. Motivation, Mental Health, and Overcoming Common Barriers


In [11]:
# If you want to elaborate on a specific section, uncomment and modify the following lines
# Choose a specific section to elaborate on

# Choose a specific topic to elaborate on
elaborate_topic = "Lifestyle and Behavioral Interventions: Evidence-Based Approaches and Emerging Trends"

# Request elaboration within the current thread
elaborate_msg = [
    {"role": "user", "content": f"Now that we have a good understanding of Patient P001's condition from the report, I'd like more detailed information specifically about '{elaborate_topic}'. Please create a focused mini-report that goes deeper into this topic, including the most recent clinical evidence, medication classes, dosing considerations, and monitoring recommendations. Focus specifically on how this applies to Patient P001's clinical profile of moderate Type 2 Diabetes (HbA1c 7.8%) and mild Hypertension (BP 135/88)."}
]

# Run the process for the elaboration using the SAME thread configuration
elaborate_response = await agent.ainvoke({"messages": elaborate_msg}, config=access_config)

# Check the response
elaborate_messages = agent.get_state(access_config).values['messages']
elaborate_messages[-1].pretty_print()

INFO:open_deep_research.multi_agent_mcp:Initializing MCP manager with 1 servers
INFO:open_deep_research.mcp_integration:Creating MCP manager with servers: ['patient_data']
INFO:open_deep_research.mcp_integration:Initializing MCP client with servers: ['patient_data']
INFO:open_deep_research.mcp_integration:Server patient_data config: transport=stdio
INFO:open_deep_research.mcp_integration:  Command: python /Users/bartolli/Projects/playgound/open_deep_research_05/open_deep_research/src/open_deep_research/example_mcp_server.py
INFO:open_deep_research.mcp_integration:Successfully loaded 3 tools from MCP servers
INFO:open_deep_research.mcp_integration:Tool 1: get_patient_vitals - 
Use this tool to retrieve current vital signs for a patient for your research.

Args:
    patient_id: Unique identifier for the patient
    
Returns:
    String containing vital measurements information

INFO:open_deep_research.mcp_integration:  Arguments: {'patient_id': {'title': 'Patient Id', 'type': 'string'}}


================================== Ai Message ==================================

Here is a focused synthesis on “Lifestyle and Behavioral Interventions: Evidence-Based Approaches and Emerging Trends” for Patient P001 (moderate T2D, HbA1c 7.8%, mild hypertension BP 135/88):

---

### Lifestyle and Behavioral Interventions: Evidence and Emerging Trends

#### 1. Core Lifestyle Interventions—Latest Evidence

**A. Dietary Interventions**
- Recent expert consensus (American College of Lifestyle Medicine, 2022) underscores structured dietary changes as foundational for improving HbA1c and lowering blood pressure.
- Effective approaches for moderate T2D with coexisting mild hypertension include:
  - Mediterranean-style, DASH, plant-based, or low-carbohydrate diets, focusing on high vegetables, whole grains, lean proteins, and reducing processed/sugary foods.
  - Sodium restriction (<2,300 mg/day, ideally <1,500 mg), which synergistically benefits both glycemic and BP control.
  - Individualiz